In [17]:
import os
mass=200
# channel="WW"
channel="numunumu"

profile='NFW'
mcinj="1122"
mcfit="1122"
process='ann'
bkgtrial=0
Ntrials = 2000
Njobs = 5
SignalSub = True


In [18]:
#############################################################################################################
#   Create .config, .submit files -> change to your own folder
#
submit_dir = '/home/tchau/code/condor_submit/TSdist'
logdir = '/scratch/tchau/DM_OscNext/MakePDF/logs/'
if not (os.path.exists(submit_dir)): os.makedirs(submit_dir)
if not (os.path.exists(logdir)): os.makedirs(logdir)

cfig_file = f'{submit_dir}/dagman.config'
curdir=os.getcwd()
script = f'{curdir}/TS_distribution.py'

cfig_file = f'{submit_dir}/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()
if SignalSub:
    sub_file = f'{submit_dir}/TSdist_SignalSubLLH.submit'
else:
    sub_file = f'{submit_dir}/TSdist_NormalLLH.submit'

with open(sub_file, 'w') as s:
    s.write(f'executable = {script}'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write(f'logdir = {logdir}'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 8GB'+'\n')
    s.write('getenv = True'+'\n')

    s.write('\n')
    if SignalSub:
        s.write('arguments = -m $(mass) -c $(channel) -p $(profile) --process $(proc) --mcinj $(mcinj) --mcfit $(mcfit) --bkgtrial $(bkgtrial)  -n $(Ntrials) -f $(file) --signalsub --GPmodel $(gpmodel) --GPinject $(gpinject)'+'\n')
    else:
        s.write('arguments = -m $(mass) -c $(channel) -p $(profile) --process $(proc) --mcinj $(mcinj) --mcfit $(mcfit) --bkgtrial $(bkgtrial)  -n $(Ntrials) -f $(file) --GPmodel $(gpmodel) --GPinject $(gpinject)'+'\n')

    s.write('queue')
    s.close()    

In [19]:
submit = '/home/tchau/code/condor_submit/TSdist/submit.sh'
path = '/home/tchau/code/condor_submit/TSdist/{}_{}'.format(channel, mass)
if not (os.path.exists(path)): os.makedirs(path)

# gpinjs = [None, 'pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']
# gpmodels = [None, 'pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']
gpinjs = [None]
gpmodels = [None]

with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for gpinj in gpinjs:
        for gpmodel in gpmodels:
            dag_file = f'{path}/TSdist_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_bkgtrial{bkgtrial}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}.dag'
            with open(dag_file, 'w') as f:
                for j in range(Njobs):
                # for j in range(10):  

                        if SignalSub:
                            f.write(f'JOB TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_bkgtrial{bkgtrial}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}' +' TSdist_SignalSubLLH.submit \n')
                        else:
                            f.write(f'JOB TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_bkgtrial{bkgtrial}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}'+' TSdist_NormalLLH.submit \n')
            
                        f.write(f'VARS TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_bkgtrial{bkgtrial}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}'+
                                f' JOBNAME="TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_bkgtrial{bkgtrial}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}"' 
                                +' mass="{}" channel="{}" profile="{}" proc="{}" bkgtrial="{}" mcinj="{}" mcfit="{}" Ntrials="{}" file="{}" gpmodel="{}" gpinject="{}" '.format(mass, channel, profile, process, bkgtrial,mcinj, mcfit, Ntrials, j, gpmodel, gpinj)+'\n')
            f.close()
            s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
            print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/numunumu_200/TSdist_SignalSubFalse_numunumu_200_NFW_ann_bkgtrial0_mcinj1122_mcfit1122_gpinjNone_gpmodelNone.dag

